In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q torch
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q pyarrow==13.0.0
!pip install -q datasets
!pip install wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
import torch
import transformers
import gc
import wandb

from transformers import TrainingArguments, DataCollatorForLanguageModeling, Trainer
from transformers.models.mistral.modeling_mistral import MistralForCausalLM
from transformers.models.llama.tokenization_llama_fast import LlamaTokenizerFast
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from peft import AutoPeftModelForCausalLM, prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

from datasets import DatasetDict, Dataset, IterableDatasetDict, IterableDataset, load_dataset, load_from_disk

from google.colab import userdata, drive

from IPython.display import HTML, display

from typing import Tuple, TypeVar, List, Dict
from typing_extensions import NoReturn

In [3]:
T = TypeVar("T")

DataPointType=DatasetDict | Dataset | IterableDatasetDict | IterableDataset
DataPointTrain =Dataset | List | T
DataPointTest =Dataset | List | T

In [4]:
# Defined in the secrets tab in Google Colab
hf_token = userdata.get('HUGGINGFACE_TOKEN')
wb_token = userdata.get('WANDB_TOKEN')
wandb.login(key=wb_token)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
model_name = "mistralai/Mistral-7B-v0.1"
new_model = "Decompose_Task_Mistral-7B"

In [6]:
wandb.init(project="fine-tune mistral 7b for decomposition", name="decompose_task")

wandb: Currently logged in as: ashmsn8665. Use `wandb login --relogin` to force relogin


In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [9]:
def get_completion(query: str, model:MistralForCausalLM, tokenizer:LlamaTokenizerFast, device:str="cuda:0") -> str:

    prompt_template = """
    Below is an instruction paired with an input that outlines a composite task. Your response should fulfill the instruction by providing a structured breakdown of the task into distinct subtasks.
    ### Question:
    {query}

    ### Answer:
    """
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1000,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0]

In [10]:
def load_model_tokenizer(model_id: str = model_name, device_map:Dict[str,int]={"": 0})->Tuple[MistralForCausalLM,LlamaTokenizerFast]:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id, quantization_config=bnb_config, device_map=device_map
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)
    return model, tokenizer

In [11]:
model, tokenizer=load_model_tokenizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [12]:
query="Generate an image and analyze the result."
result = get_completion(query=query, model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> 
    Below is an instruction paired with an input that outlines a composite task. Your response should fulfill the instruction by providing a structured breakdown of the task into distinct subtasks.
    ### Question:
    Generate an image and analyze the result.

    ### Answer:
    </s>
The generated image is a 128x128x3 tensor.


### Processing
It is possible for this image to be generated using a Generative Adversarial Network.
Below is an illustration of the different steps that would be performed using a Generative Adversarial Network in order for the image to be generated.

1. The first step is to generate a random vector of size 100 using the Random method, and then feed it as input to the Generative Adversarial Network (GAN) in order to produce a fake image.
2. The next step is to train the GAN using the fake image and some real images.
3. Then use the Trainer method to train the Generator and Discriminator
4. The resulting fake images will be displayed alongside the real i

In [13]:
def custom_dataset_load(data_dir: str)->DataPointType:
    data = load_dataset(data_dir, split="train")
    return data

In [14]:
data=custom_dataset_load('/content/drive/MyDrive/data')

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
df = data.to_pandas()
df.head(5)

,output,instruction
0,1. Generate a detailed script covering various...,Generate a podcast episode script on modern te...
1,1. Generate a digital painting of a futuristic...,Create a digital painting of a futuristic city...
2,1. Produce a video tutorial showcasing the ste...,Develop a video tutorial on how to bake a choc...
3,1. Generate an instrumental music piece that c...,Compose a piece of instrumental music inspired...
4,1. Design a 3D model of an eco-friendly house....,Create a 3D model of an eco-friendly house and...


In [16]:
def generate_prompt(data_point: Dict[str, str]) -> str:
    """Generate input text based on a prompt, task instruction, (context info.), and answer.

    Args:
        data_point (Dict[str, str]): Data point

    Returns:
        str: prompt
    """
    text = (
        "Below is an instruction that describes a composite task."
        " Your response should fulfill the instruction by providing a structured breakdown of the composite task into distinct subtasks.\n\n"
    )
    text += f'### Instruction:\n{data_point["instruction"]}\n\n'
    text += f'### Response:\n{data_point["output"]}'
    return text


def add_column(data:DataPointType, column_name: str = "prompt")->DataPointType:
    """Add the "prompt" column in the dataset"""
    text_column = [generate_prompt(data_point) for data_point in data]
    data = data.add_column(column_name, text_column)
    return data

In [17]:
new_data=add_column(data)

In [18]:
new_df = new_data.to_pandas()
new_df.head(5)

,output,instruction,prompt
0,1. Generate a detailed script covering various...,Generate a podcast episode script on modern te...,Below is an instruction that describes a compo...
1,1. Generate a digital painting of a futuristic...,Create a digital painting of a futuristic city...,Below is an instruction that describes a compo...
2,1. Produce a video tutorial showcasing the ste...,Develop a video tutorial on how to bake a choc...,Below is an instruction that describes a compo...
3,1. Generate an instrumental music piece that c...,Compose a piece of instrumental music inspired...,Below is an instruction that describes a compo...
4,1. Design a 3D model of an eco-friendly house....,Create a 3D model of an eco-friendly house and...,Below is an instruction that describes a compo...


In [19]:
def split_train_test_dataset(data:DataPointType, split_size: float = 0.1)->Tuple[DataPointTrain,DataPointTest]:
    data = data.train_test_split(test_size=split_size)
    train_data = data["train"]
    test_data = data["test"]
    return train_data, test_data

In [20]:
def shuffle_tokenize_batch(data:DataPointType, tokenizer:LlamaTokenizerFast)->DataPointType:
    data = data.shuffle(seed=1234)  # Shuffle dataset here
    data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
    return data

In [21]:
processed_data=shuffle_tokenize_batch(new_data,tokenizer)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
processed_df = processed_data.to_pandas()
processed_df.head(5)

,output,instruction,prompt,input_ids,attention_mask
0,1. Generate a script that covers the landmark'...,Produce a documentary short on the history of ...,Below is an instruction that describes a compo...,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1) Design a meme that communicates the importa...,Create a meme about the importance of voting a...,Below is an instruction that describes a compo...,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1. Produce an audio documentary that covers th...,Record an audio documentary on the history of ...,Below is an instruction that describes a compo...,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1. Generate a synthetic voice that emulates th...,Develop a synthetic voice that sounds like a f...,Below is an instruction that describes a compo...,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1. Research topics on urban wildlife. 2. Outli...,"Create a documentary script on urban wildlife,...",Below is an instruction that describes a compo...,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [23]:
train_data, test_data=split_train_test_dataset(processed_data)

In [24]:
print(len(train_data),len(test_data))

900 100


In [25]:
def print_trainable_parameters(model:MistralForCausalLM)->NoReturn:
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    # Iterate through model parameters
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    # Print the number of trainable parameters, total parameters, and the percentage of trainable parameters
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [26]:
# Define a configuration for the LoRA (Learnable Requantization Activation) method
lora_config = LoraConfig(
    r=8,                                  # Number of quantization levels
    lora_alpha=32,                         # Hyperparameter for LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # Modules to apply LoRA to
    lora_dropout=0.05,                     # Dropout probability
    bias="none",                           # Type of bias
    task_type="CAUSAL_LM"                  # Task type (in this case, Causal Language Modeling)
)

In [27]:
# Define a configuration for the training
training_args=TrainingArguments(
        per_device_train_batch_size=4,        # Batch size per device during training
        gradient_accumulation_steps=4,        # Number of gradient accumulation steps
        gradient_checkpointing=True,
        warmup_ratio = 0.03 ,                 # Number of warm-up steps for learning rate
        max_steps=200,                        # Maximum number of training steps
        learning_rate=5e-5,                   # Learning rate
        lr_scheduler_type="cosine",           # cosine learning rate scheduler
        bf16=True,                            # Enable mixed-precision training
        logging_steps=1,                      # Logging frequency during training
        output_dir=new_model,                 # Directory to save output files
        optim="paged_adamw_32bit",            # Optimizer type
        save_strategy="epoch",                # Strategy for saving checkpoints
        # push_to_hub=True                    # Push to the Hugging Face model hub
        report_to="wandb",
        do_eval=True,
        evaluation_strategy="steps",
        eval_steps=5,
    )

In [28]:
def prepare_model_for_qlora_training(model:MistralForCausalLM,lora_config)->MistralForCausalLM:
  # Enable gradient checkpointing for the model
  model.gradient_checkpointing_enable()

  # Prepare the model for k-bit training using the "prepare_model_for_kbit_training" function
  model = prepare_model_for_kbit_training(model)

  # Get a model with LoRA applied to it using the defined configuration
  model = get_peft_model(model, lora_config)

  return model

In [29]:
model = prepare_model_for_qlora_training(model,lora_config)

In [30]:
# Print the number of trainable parameters in the model after applying LoRA
print_trainable_parameters(model)

trainable params: 6815744 || all params: 3758886912 || trainable%: 0.18132346515244138


In [31]:
def qlora_training(training_args,model:MistralForCausalLM,tokenizer:LlamaTokenizerFast,train_data,test_data):
  # Set the pad_token of the tokenizer to be the same as the eos_token
  tokenizer.pad_token = tokenizer.eos_token

  # Create a trainer for fine-tuning a model
  trainer = Trainer(
      model=model,  # The model to be trained
      train_dataset=train_data,  # Training dataset
      eval_dataset=test_data,    # Evaluation dataset
      args=training_args,
      data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False), # Data collator for language modeling task
  )

  model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

  trainer.train()

  return trainer, tokenizer, model

In [32]:
# Start QLORA fine-tuning
trainer, tokenizer, model = qlora_training(training_args,model,tokenizer,train_data,test_data)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
5,2.017100,1.982379
10,1.537200,1.407969
15,1.052100,0.970603
20,0.851000,0.836154
25,0.808100,0.769679
30,0.745500,0.740560
35,0.741700,0.719056
40,0.724700,0.706709
45,0.718400,0.698502
50,0.719800,0.688504


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [33]:
# Good practice to end your run
wandb.finish()

eval/loss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▃▂▂▂▁▁▂▂▂▃▂▂▂▂▄▂▂▂▂▂▂▂▂▁▃▂▂▂▃▂▃▃▂▁█▃▃▄▄▆
eval/samples_per_second,▆▇▇▇▇█▇▇▇▆▇▇▇▇▅▇▇▇▇▇▇▇▇█▆▇▇▇▆▇▆▆▇█▁▆▅▅▅▃
eval/steps_per_second,▆▇▇▇▇█▇▇▇▆▇▇▇▇▅▇▇▇▇▇▇▇▇█▆▇▇▇▆▇▆▆▇█▁▆▆▅▅▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▅▅▄▂▂▂▁▂▂▂▁▁▁▁▂▂▂▁▁▁▁▁▁▂▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂
train/learning_rate,▃███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▆▄▃▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [34]:
# Save artifacts
trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del trainer, model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Push them to the HF Hub
#model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
#tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('Decompose_Task_Mistral-7B/tokenizer_config.json',
 'Decompose_Task_Mistral-7B/special_tokens_map.json',
 'Decompose_Task_Mistral-7B/tokenizer.model',
 'Decompose_Task_Mistral-7B/added_tokens.json',
 'Decompose_Task_Mistral-7B/tokenizer.json')

In [35]:
def create_model_inputs(query:str,device:str="cuda:0"):
  prompt_template = """
      Below is an instruction paired with an input that outlines a composite task. Your response should fulfill the instruction by providing a structured breakdown of the task into distinct subtasks.
      ### Question:
      {query}

      ### Answer:
      """


  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to(device)
  return model_inputs

In [36]:
def custom_inference(model_inputs,model):
  generated_ids = model.generate(
          **model_inputs,
          max_new_tokens=1000,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

In [37]:
model_inputs = create_model_inputs("Generate an image and analyze the result.")

In [38]:
model=model.to("cuda:0")

In [39]:
response = custom_inference(model_inputs,model)
print(response)

<s> 
      Below is an instruction paired with an input that outlines a composite task. Your response should fulfill the instruction by providing a structured breakdown of the task into distinct subtasks.
      ### Question:
      Generate an image and analyze the result.

      ### Answer:
      1. Generate an interesting and engaging image using a generative algorithm. 2. Analyze the generated image by assessing its visual appeal and understanding. 3. Provide feedback on the image based on the analysis. 4. Compare the generated image with others of similar concepts to see its uniqueness. 5. Evaluate the image for common or unusual elements. 6. Determine the image's impact and likelihood of generating a strong emotional response. 7. Predict how the image will be received by various audiences. 8. Discuss the image in relation to its potential use for marketing, art, or education. 9. Compare the image to other works within the same genre or style. 10. Evaluate the artistic merit and tec

In [40]:
# Prepare evaluation prompts
eval_prompts = [
    f"""
    Below is an instruction paired with an input that outlines a composite task. Your response should fulfill the instruction by providing a structured breakdown of the task into distinct subtasks.
    ### Question:
    {sample["instruction"]}

    ### Answer:
    """ for sample in test_data
]

In [41]:
eval_prompts[25]

'\n    Below is an instruction paired with an input that outlines a composite task. Your response should fulfill the instruction by providing a structured breakdown of the task into distinct subtasks.\n    ### Question:\n    Create a lullaby for children, including the lyrics and the melody.\n\n    ### Answer:\n    '

In [42]:
# Get references / Ground Truth the model will be evaluated against
references = [sample["output"] for sample in test_data]

In [43]:
references[25]

'1. Write the lyrics for the lullaby. 2. Compose the melody for the lullaby.'

In [44]:
model_inputs_eval = create_model_inputs(test_data[25]["instruction"])

In [45]:
response_eval = custom_inference(model_inputs_eval,model)
print(response_eval)

<s> 
      Below is an instruction paired with an input that outlines a composite task. Your response should fulfill the instruction by providing a structured breakdown of the task into distinct subtasks.
      ### Question:
      Create a lullaby for children, including the lyrics and the melody.

      ### Answer:
      1. Write a calming lullaby for children, emphasizing themes of rest and comfort. 2. Compose a melody that is soothing and suitable for lullabies. 3. Analyze the lyrics and melody for their effectiveness in calming children down.

      Based on the Question, it asks to create a lullaby for children, including the lyrics and the melody. The Answer should outline a composite task by providing a structured breakdown of the Question into distinct subtasks.

      ### Structured Answer (Composite Task):
      1. Write a lullaby specifically targeted towards children. 2. Compose a soothing melody for the lullaby. 3. Combine the lullaby's lyrics and melody. 4. Perform or rec

In [46]:
import json
from tqdm import tqdm

N_test = len(test_data)
responses_eval=[]
for i in tqdm(range(N_test)):
  model_inputs_eval = create_model_inputs(test_data[i]["instruction"])
  response_eval = custom_inference(model_inputs_eval,model)
  responses_eval.append(response_eval)

100%|██████████| 100/100 [10:01<00:00,  6.02s/it]


In [47]:
with open('eval_responses.json', 'w') as json_file:
    json.dump(responses_eval, json_file, indent=4)

In [ ]:
with open('eval_responses.json', 'r') as json_file:
    results=json.load(json_file)

print(results[0])

In [ ]:
import re

for i in tqdm(range(N_test)):
  text = results[i]
  pattern = "### Answer:"

  # Using re.search() to find the pattern
  match = re.search(pattern, text)
  if match:
      # Extracting everything after the match
      start_position = match.end()  # The end of the match's position
      following_text = text[start_position:]
      print(following_text)
  else:
      print("Pattern not found.")